In [1]:
import pandas as pd
import numpy as np
import math
from io import StringIO

#NOTES
#
#Get rid of info after comma in names
#

In [2]:
def squeeze_nan_left(x):
            original_columns = x.index.tolist()
            squeezed = x.dropna()
            squeezed.index = [original_columns[n] for n in range(squeezed.count())]
            return squeezed.reindex(original_columns, fill_value=np.nan)

In [3]:
def SCT_table_format(SCT):
            #SCT = SCT.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

            for i in range(0,SCT.shape[0]):
                for j in range(0,SCT.shape[1]):
                    if type(SCT.iloc[i,j]) == str:
                        if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                            SCT.iloc[i,j] = 0
                        elif '(' in SCT.iloc[i,j]:
                            tempstr = SCT.iloc[i,j].split("(")
                            tempstr2 = tempstr[0]
                            SCT.iloc[i,j] = tempstr2
                        elif len(SCT.iloc[i,j]) < 2:
                            SCT.iloc[i,j] = np.nan             

            SCT = SCT.replace('', np.nan)
            SCT.iloc[:,0] = SCT.iloc[:,0].fillna(value = '')
            SCT = SCT.fillna(value = '')

            for i in range(0,SCT.shape[0]):
                if SCT.iloc[i, SCT.shape[1]-1] == '':
                    rightslip = ['',SCT.iloc[i]]
                    rightslip = rightslip[:-1]
                    SCT.iloc[i] = rightslip

            SCT.replace('',np.nan,inplace = True)
            SCT = SCT.dropna()

            SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 1)

            return SCT

In [79]:
#Everi - https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm
#MNTA - https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm

url = 'https://www.sec.gov/Archives/edgar/data/1410939/000104746917002856/a2231921zdef14a.htm#ds46201_summary_compensation_table'

filing = pd.read_html(url)
#filing[:]
#168 for Everi
#128 for MNTA
#20 for OPHT

ticker = 'OPHT'
sctind = 20
SCTo = filing[sctind]
SCTo

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name and principal position,NaN,Year,NaN,Salary ($),NaN,Bonus ($)(1),NaN,Stock Awards ($)(2),NaN,...,Total ($),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"David R. Guyer, M.D.(5)",NaN,NaN,2016.0,NaN,NaN,625200,NaN,NaN,—,...,NaN,NaN,325104,NaN,NaN,10405,NaN,NaN,8700989.0,NaN
3,Chief Executive,NaN,NaN,2015.0,NaN,NaN,600000,NaN,NaN,—,...,NaN,NaN,486000,NaN,NaN,18758,NaN,NaN,4866882.0,NaN
4,Officer,NaN,NaN,2014.0,NaN,NaN,545000,NaN,NaN,—,...,NaN,NaN,523200,NaN,NaN,8000,NaN,NaN,5060110.0,NaN
5,"Samir C. Patel, M.D.(6)",NaN,NaN,2016.0,NaN,NaN,591231,NaN,NaN,—,...,NaN,NaN,325104,NaN,NaN,31455,NaN,NaN,8688070.0,NaN
6,Former President,NaN,NaN,2015.0,NaN,NaN,486450,NaN,NaN,—,...,NaN,NaN,361190,NaN,NaN,9038,NaN,NaN,7681275.0,NaN
7,NaN,NaN,NaN,2014.0,NaN,NaN,470000,NaN,NaN,60000,...,NaN,NaN,376000,NaN,NaN,—,NaN,NaN,4093128.0,NaN
8,Glenn P. Sblendorio Sblendorio(7),NaN,NaN,2016.0,NaN,NaN,373154,NaN,NaN,100000,...,NaN,NaN,163350,NaN,NaN,98356,NaN,NaN,8878904.0,NaN
9,President and,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Table Formatting tests

#initial format, take out nan rows/cols and squeeze nan left
SCT = SCTo.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

#secondary format - fill dashes with 0s, remove symbols and parenthesis objects
for i in range(0,SCT.shape[0]):
    for j in range(0,SCT.shape[1]):
        if type(SCT.iloc[i,j]) == str:
            if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                SCT.iloc[i,j] = 0
            elif '(' in SCT.iloc[i,j]:
                tempstr = SCT.iloc[i,j].split("(")
                tempstr2 = tempstr[0]
                SCT.iloc[i,j] = tempstr2
            elif len(SCT.iloc[i,j]) < 2:
                SCT.iloc[i,j] = np.nan             

SCT = SCT.reset_index(drop=True)


#correct for years that are bumped too far left
for i in range(0,SCT.shape[0]):
    if pd.isnull(SCT.iloc[i, SCT.shape[1]-1]):
        rightslip = SCT.iloc[i].tolist()
        rightslip.insert(0,'HOLD')
        SCT.iloc[i] = rightslip[:-1]
        

#correct for positions moved over to the right
for i in range(0,SCT.shape[0]):
    if SCT.iloc[i].isnull().values.any():
        leftslip = SCT.iloc[i].tolist()
        leftslip.insert(SCT.shape[1]-1,np.nan)
        leftslip.remove("HOLD")
        SCT.iloc[i] = leftslip
        
SCT.replace(to_replace = 'HOLD', value = '', inplace = True)

#don't do this, screws up Name df below
#SCT = SCT.fillna(value = '')

#update column names
SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 0)                

SCT = SCT.reset_index(drop=True)
SCT


,Name and principal position,Year,Salary,Bonus,Stock Awards,Option Awards,Non- Equity Incentive Plan Compensation,All Other Compensation,Total
0,"David R. Guyer, M.D.",2016,625200,0,3291750,4448530,325104,10405,8.70099e+06
1,Chief Executive,2015,600000,0,1048800,2713323,486000,18758,4.86688e+06
2,Officer,2014,545000,0,0,3983910,523200,8000,5.06011e+06
3,"Samir C. Patel, M.D.",2016,591231,0,3291750,4448530,325104,31455,8.68807e+06
4,Former President,2015,486450,0,4511600,2312997,361190,9038,7.68128e+06
5,,2014,470000,60000,0,3187128,376000,0,4.09313e+06
6,Glenn P. Sblendorio Sblendorio,2016,373154,100000,3887705,4256339,163350,98356,8.8789e+06
7,President and,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Former Chief Operating Officer, Chief Financia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Barbara A. Wood,2016,404620,0,632016,899190,129478,17337,2.08264e+06


In [64]:
#creating NEO name/position lists - done after table format
#will need to grab posisitons from SCTo
#consider making a version that chops off duplicate names, middle initial, endings like M.D.

NEO_Names = []
#max_year = 

for i in range(0,SCT.shape[0]):
    if SCT.iloc[i,1] == SCT.iloc[:,1].max():
        NEO_Names.append(SCT.iloc[i,0])
        
        
NEO_Combine = pd.DataFrame(NEO_Names)
NEO_Combine.columns = ['Name']
SCT_Positions = SCT.iloc[:,0]
NEO_Combine['Position'] = np.nan
NEO_Combine

,Name,Position
0,"David R. Guyer, M.D.",NaN
1,"Samir C. Patel, M.D.",NaN
2,Glenn P. Sblendorio Sblendorio,NaN
3,Barbara A. Wood,NaN
4,Henric B. Bjarke,NaN
5,Michael G. Atieh,NaN


In [65]:
NEO_Combine['Position'] = ''

k = 0

for i in range(1, len(SCT_Positions)):
    if SCT_Positions[i] in NEO_Names:
        k = k + 1
        i = i + 1
    else:
        NEO_Combine.iloc[k,1] = NEO_Combine.iloc[k,1] + ' ' + SCT_Positions[i]
        i = i + 1


NEO_Combine

,Name,Position
0,"David R. Guyer, M.D.",Chief Executive Officer
1,"Samir C. Patel, M.D.",Former President
2,Glenn P. Sblendorio Sblendorio,"President and Former Chief Operating Officer,..."
3,Barbara A. Wood,"Senior Vice President, General Counsel"
4,Henric B. Bjarke,"Former Senior Vice President, Chief Commercia..."
5,Michael G. Atieh,"Former Executive Vice President, Chief Financ..."


In [63]:
NEO_Combine.iloc[5,1]

' Former Executive Vice President, Chief Financial Officer'

In [107]:
#Putting it all together

#throw the warning for copying
pd.options.mode.chained_assignment = None  # default='warn'


SCT_Final = SCT.iloc[:,1:].dropna().reset_index(drop=True)
SCT_Final.insert(loc = 0, column = 'Ticker', value = ticker)
SCT_Final.insert(loc = 1, column = 'Name', value = np.nan)
SCT_Final.insert(loc = 2, column = 'Position', value = np.nan)

k = 0

for i in range(0, SCT_Final.shape[0]):
    if (i>0) and (SCT_Final['Year'][i] >= SCT_Final['Year'][i-1]):
        k = k + 1
    SCT_Final['Name'][i] = NEO_Combine['Name'][k]
    SCT_Final['Position'][i] = NEO_Combine['Position'][k]
    


SCT_Final

,Ticker,Name,Position,Year,Salary,Bonus,Stock Awards,Option Awards,Non- Equity Incentive Plan Compensation,All Other Compensation,Total
0,OPHT,"David R. Guyer, M.D.",Chief Executive Officer,2016,625200,0,3291750,4448530,325104,10405,8.70099e+06
1,OPHT,"David R. Guyer, M.D.",Chief Executive Officer,2015,600000,0,1048800,2713323,486000,18758,4.86688e+06
2,OPHT,"David R. Guyer, M.D.",Chief Executive Officer,2014,545000,0,0,3983910,523200,8000,5.06011e+06
3,OPHT,"Samir C. Patel, M.D.",Former President,2016,591231,0,3291750,4448530,325104,31455,8.68807e+06
4,OPHT,"Samir C. Patel, M.D.",Former President,2015,486450,0,4511600,2312997,361190,9038,7.68128e+06
5,OPHT,"Samir C. Patel, M.D.",Former President,2014,470000,60000,0,3187128,376000,0,4.09313e+06
6,OPHT,Glenn P. Sblendorio Sblendorio,"President and Former Chief Operating Officer,...",2016,373154,100000,3887705,4256339,163350,98356,8.8789e+06
7,OPHT,Barbara A. Wood,"Senior Vice President, General Counsel",2016,404620,0,632016,899190,129478,17337,2.08264e+06
8,OPHT,Barbara A. Wood,"Senior Vice President, General Counsel",2015,390940,0,250800,667211,211110,8000,1.52806e+06
9,OPHT,Barbara A. Wood,"Senior Vice President, General Counsel",2014,372311,60000,0,335815,240000,8000,1.01613e+06
